In [1]:
import numpy as np
import pandas
import os
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import gaussian_kde
import sys
from tqdm import tqdm
import torch

# Set up path to import from src
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

# Import pokie from pokie.py
from pokie import pokie, get_device

# Section 3.2: Analyzing distribution shifts

# Sample from GMM

In [2]:
def generate_samples_from_gmm(means, covariances, n_samples):
    """
    means:       shape (n_components, n_dimensions)
    covariances: shape (n_components, n_dimensions),
                 each row is the diagonal of the covariance matrix for that component
    n_samples:   number of samples to generate in total
    """
    n_components, n_dimensions = means.shape
    samples = np.zeros((n_samples, n_dimensions))
    # Assume uniform mixing weights for simplicity
    component_choices = np.random.choice(
        n_components, size=n_samples, p=np.ones(n_components)/n_components
    )

    for i, comp in enumerate(component_choices):
        # Use np.diag(...) so each component's covariance is diagonal
        cov = np.diag(covariances[comp])
        samples[i, :] = np.random.multivariate_normal(means[comp], cov)
    return samples

# 2 Dimension

In [3]:
# Set up the GMM parameters
n_components = 20
n_dimensions = 2
n_truth_samples = 5000  # Number of ground truth samples
num_posterior_samples = 5000  # Number of posterior samples per truth
epsilon = 1e-3  # Small value to avoid division by zero
curr_num_runs = 100

# Initialize random means and covariances for the GMM components
means = np.random.rand(n_components, n_dimensions) * 10  # Random means between 0 and 10
covariances = np.random.rand(n_components, n_dimensions) + epsilon  # Ensure strictly positive variance

# Generate the truth data (no shift)
truth_data = generate_samples_from_gmm(means, covariances, n_truth_samples)

# Generate the models with different shift magnitudes from -10 to 10 along the diagonal
shift_magnitudes = np.arange(-10, 11, 1)  # Shift values from -10 to 10  # Shift values from -10 to 10
# shift_magnitudes = np.arange(-3, 4, 1)  # Shift values from -3 to 3  # Shift values from -10 to 10
shift_magnitudes = [-6, -3, 0, 3, 6]  # Example shift magnitudes for testing
num_models = len(shift_magnitudes)

print(f'Shift magnitudes: {shift_magnitudes}')

# Generate shifted GMMs once per model
models_base = np.zeros((num_models, num_posterior_samples, n_dimensions))  # Shape: (21, 500, 2)

for i, shift in enumerate(tqdm(shift_magnitudes, desc=f'Generating Shifted GMMs')):
    models_base[i] = generate_samples_from_gmm(means, covariances, num_posterior_samples) + np.ones(n_dimensions) * shift

# Now, expand to (num_models, num_truth_samples, num_posterior_samples, dimensions)
models = np.repeat(models_base[:, np.newaxis, :, :], n_truth_samples, axis=1)

# Validate shape
assert models.shape == (num_models, n_truth_samples, num_posterior_samples, n_dimensions), "Shape mismatch!"

epsilon = 1e-10  # Small value to avoid division by zero

# Get min and max from truth_data (per dimension)
low = np.min(truth_data, axis=0, keepdims=True)  # Shape: (1, n_dimensions)
high = np.max(truth_data, axis=0, keepdims=True)  # Shape: (1, n_dimensions)

# Normalize truth_data
truth_data_normalized = (truth_data - low) / (high - low + epsilon)

# Normalize models (loop over each shift magnitude)
models_normalized = np.zeros_like(models)
for m_idx in range(num_models):
    models_normalized[m_idx] = (models[m_idx] - low) / (high - low + epsilon)

Shift magnitudes: [-6, -3, 0, 3, 6]


Generating Shifted GMMs: 100%|██████████| 5/5 [00:00<00:00,  7.24it/s]


In [4]:
# Identify your device (CUDA > MPS > CPU)
device = get_device()
print("Using device:", device)

# Convert to torch Tensors on the chosen device
truth_data_normalized   = torch.tensor(truth_data_normalized, dtype=torch.float32, device=device)
models_normalized = torch.tensor(models_normalized,   dtype=torch.float32, device=device)

pokie_score = pokie(
    truth_data_normalized, models_normalized, num_runs=curr_num_runs
)

pokie_score = pokie_score.cpu().numpy()

print('\nShift Magnitudes:', shift_magnitudes)
print("Pokie Score:", pokie_score)

Using device: mps


Pokie MC runs: 100%|██████████| 100/100 [00:06<00:00, 16.43it/s]



Shift Magnitudes: [-6, -3, 0, 3, 6]
Pokie Score: [0.5920529  0.64539456 0.6670515  0.6426327  0.59763235]


# 20 Dimensions

In [5]:
# Set up the GMM parameters
n_components = 20
n_dimensions = 20
n_truth_samples = 500  # Number of ground truth samples
num_posterior_samples = 500  # Number of posterior samples per truth
epsilon = 1e-3  # Small value to avoid division by zero
curr_num_runs = 100

# Initialize random means and covariances for the GMM components
means = np.random.rand(n_components, n_dimensions) * 10  # Random means between 0 and 10
covariances = np.random.rand(n_components, n_dimensions) + epsilon  # Ensure strictly positive variance

# Generate the truth data (no shift)
truth_data = generate_samples_from_gmm(means, covariances, n_truth_samples)

# Generate the models with different shift magnitudes from -10 to 10 along the diagonal
# shift_magnitudes = np.arange(-10, 11, 1)  # Shift values from -10 to 10  # Shift values from -10 to 10
shift_magnitudes = [-6, -3, 0, 3, 6]  # Example shift magnitudes for testing
num_models = len(shift_magnitudes)

print(f'Shift magnitudes: {shift_magnitudes}')

# Generate shifted GMMs once per model
models_base = np.zeros((num_models, num_posterior_samples, n_dimensions))  # Shape: (21, 500, 2)

for i, shift in enumerate(tqdm(shift_magnitudes, desc=f'Generating Shifted GMMs')):
    models_base[i] = generate_samples_from_gmm(means, covariances, num_posterior_samples) + np.ones(n_dimensions) * shift

# Now, expand to (num_models, num_truth_samples, num_posterior_samples, dimensions)
models = np.repeat(models_base[:, np.newaxis, :, :], n_truth_samples, axis=1)

# Validate shape
assert models.shape == (num_models, n_truth_samples, num_posterior_samples, n_dimensions), "Shape mismatch!"

epsilon = 1e-10  # Small value to avoid division by zero

# Get min and max from truth_data (per dimension)
low = np.min(truth_data, axis=0, keepdims=True)  # Shape: (1, n_dimensions)
high = np.max(truth_data, axis=0, keepdims=True)  # Shape: (1, n_dimensions)

# Normalize truth_data
truth_data_normalized = (truth_data - low) / (high - low + epsilon)

# Normalize models (loop over each shift magnitude)
models_normalized = np.zeros_like(models)
for m_idx in range(num_models):
    models_normalized[m_idx] = (models[m_idx] - low) / (high - low + epsilon)

Shift magnitudes: [-6, -3, 0, 3, 6]


Generating Shifted GMMs: 100%|██████████| 5/5 [00:00<00:00, 56.54it/s]


In [6]:
# Identify your device (CUDA > MPS > CPU)
device = get_device()
print("Using device:", device)

# Convert to torch Tensors on the chosen device
truth_data_normalized   = torch.tensor(truth_data_normalized, dtype=torch.float32, device=device)
models_normalized = torch.tensor(models_normalized,   dtype=torch.float32, device=device)

pokie_score = pokie(
    truth_data_normalized, models_normalized, num_runs=curr_num_runs
)

# Convert results, calibrated, n_over_N_vals back to numpy arrays
pokie_score = pokie_score.cpu().numpy()

print('\nShift Magnitudes:', shift_magnitudes)
print("Pokie Score:", pokie_score)

Using device: mps


Pokie MC runs: 100%|██████████| 100/100 [00:01<00:00, 88.36it/s] 


Shift Magnitudes: [-6, -3, 0, 3, 6]
Pokie Score: [0.50535846 0.6023627  0.66835594 0.5774247  0.5015557 ]


# 100 Dimensions

In [7]:
# Set up the GMM parameters
n_components = 20
n_dimensions = 100
n_truth_samples = 500  # Number of ground truth samples
num_posterior_samples = 500  # Number of posterior samples per truth
epsilon = 1e-3  # Small value to avoid division by zero
curr_num_runs = 100

# Initialize random means and covariances for the GMM components
means = np.random.rand(n_components, n_dimensions) * 10  # Random means between 0 and 10
covariances = np.random.rand(n_components, n_dimensions) + epsilon  # Ensure strictly positive variance

# Generate the truth data (no shift)
truth_data = generate_samples_from_gmm(means, covariances, n_truth_samples)

# Generate the models with different shift magnitudes from -10 to 10 along the diagonal
# shift_magnitudes = np.arange(-10, 11, 1)  # Shift values from -10 to 10  # Shift values from -10 to 10
shift_magnitudes = [-6, -3, 0, 3, 6]  # Example shift magnitudes for testing
num_models = len(shift_magnitudes)

print(f'Shift magnitudes: {shift_magnitudes}')

# Generate shifted GMMs once per model
models_base = np.zeros((num_models, num_posterior_samples, n_dimensions))  # Shape: (21, 500, 2)

for i, shift in enumerate(tqdm(shift_magnitudes, desc=f'Generating Shifted GMMs')):
    models_base[i] = generate_samples_from_gmm(means, covariances, num_posterior_samples) + np.ones(n_dimensions) * shift

# Now, expand to (num_models, num_truth_samples, num_posterior_samples, dimensions)
models = np.repeat(models_base[:, np.newaxis, :, :], n_truth_samples, axis=1)

# Validate shape
assert models.shape == (num_models, n_truth_samples, num_posterior_samples, n_dimensions), "Shape mismatch!"

epsilon = 1e-10  # Small value to avoid division by zero

# Get min and max from truth_data (per dimension)
low = np.min(truth_data, axis=0, keepdims=True)  # Shape: (1, n_dimensions)
high = np.max(truth_data, axis=0, keepdims=True)  # Shape: (1, n_dimensions)

# Normalize truth_data
truth_data_normalized = (truth_data - low) / (high - low + epsilon)

# Normalize models (loop over each shift magnitude)
models_normalized = np.zeros_like(models)
for m_idx in range(num_models):
    models_normalized[m_idx] = (models[m_idx] - low) / (high - low + epsilon)

Shift magnitudes: [-6, -3, 0, 3, 6]


Generating Shifted GMMs: 100%|██████████| 5/5 [00:02<00:00,  1.99it/s]


In [8]:
# Identify your device (CUDA > MPS > CPU)
device = get_device()
print("Using device:", device)

# Convert to torch Tensors on the chosen device
truth_data_normalized   = torch.tensor(truth_data_normalized, dtype=torch.float32, device=device)
models_normalized = torch.tensor(models_normalized,   dtype=torch.float32, device=device)

pokie_score = pokie(
    truth_data_normalized, models_normalized, num_runs=curr_num_runs
)

# Convert results, calibrated, n_over_N_vals back to numpy arrays
pokie_score = pokie_score.cpu().numpy()

print('\nShift Magnitudes:', shift_magnitudes)
print("Pokie Score:", pokie_score)

Using device: mps


Pokie MC runs: 100%|██████████| 100/100 [00:02<00:00, 36.30it/s]



Shift Magnitudes: [-6, -3, 0, 3, 6]
Pokie Score: [0.49919492 0.5186664  0.66873    0.5067975  0.50114316]
